### Extract and Transform

Se convirtio el dataset `movies_dataset.csv` a formato `parquet` desde `scripts/convert_csv_to_parquet.py` con las siguientes modificaciones:

- **Columnas eliminadas**: `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`
- **Valores nulos rellenados**:
  - `revenue`: 0
  - `budget`: 0
- **Conversión de tipos**:
  - `popularity`: `float64`
  - `budget`: `int64`
  - `id`: `int64`

In [482]:
import pandas as pd

df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\movies_dataset.parquet')
df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy...",30000000.0,"[{'id': 16, 'name': 'Anima...",862.0,en,"Led by Woody, Andy's toys ...",21.946943,[{'name': 'Pixar Animation...,"[{'iso_3166_1': 'US', 'nam...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name...",Released,None,Toy Story,7.7,5415.0
1,None,65000000.0,"[{'id': 12, 'name': 'Adven...",8844.0,en,When siblings Judy and Pet...,17.015539,[{'name': 'TriStar Picture...,"[{'iso_3166_1': 'US', 'nam...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name...",Released,Roll the dice and unleash ...,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Gr...",0.0,"[{'id': 10749, 'name': 'Ro...",15602.0,en,A family wedding reignites...,11.712900,"[{'name': 'Warner Bros.', ...","[{'iso_3166_1': 'US', 'nam...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name...",Released,Still Yelling. Still Fight...,Grumpier Old Men,6.5,92.0


Eliminar valores nulos de `release_date`

In [483]:
df = df.dropna(subset=['release_date'])

formato `AAAA-mm-dd` en fechas, y creacion de columna `release_year` para el año de estreno.

In [484]:
# Identificar las filas con valores incorrectos
incorrect_format = df[~df['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
# Muestra las filas con formato incorrecto.
incorrect_format

# Ya que los errores no pueden ser corregidos manualmente por falta de informacion
# Y ademas las demas filas parecieran incorrectas y con muchos Nan. Se procede a eliminar estos datos.

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19730,0.065736,NaN,[{'name': 'Carousel Produc...,NaN,104.0,Released,NaN,False,6.0,1,0.0,NaN,None,None,None,None,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id':...",NaN,68.0,Released,NaN,False,7.0,12,0.0,NaN,None,None,None,None,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media',...",NaN,82.0,Released,NaN,False,4.3,22,0.0,NaN,None,None,None,None,NaN,NaN


In [485]:
# Se borraron esas 3 filas del df.
df.drop(incorrect_format.index, inplace=True)

In [486]:
# Ahora si pasamos a datetime con formato aaaa-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

# Creacion columna release_year y muestra. 
df['release_year'] = df['release_date'].dt.year
df[['release_date','release_year']].sample(5)


,release_date,release_year
45397,2004-09-06,2004
27228,1988-09-22,1988
35109,2011-09-16,2011
10590,2005-01-01,2005
18161,1944-09-08,1944


crear columna  `return` dividiendo  `revenue` / `budget`, cuando no hay datos para calcularlo, tomar valor 0.



In [487]:
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0, axis=1)

- `belongs_to_collection `,  `production_companies ` ,  `genres ` ,  `production_countries`,  `spoken_languages` están anidados.
- deberán desanidarlos O buscar la manera de acceder sin desanidarlos.

### Empecemos por `belongs_to_collection `

In [488]:
# Esta columna contiene 40888 datos nulos y sus formatos en str . 
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40888
<class 'str'>          4488
Name: count, dtype: int64

In [489]:
# Por lo tanto crearemos una nueva columna que muestre su tipo de dato
df['belongs_to_collection_type'] = df['belongs_to_collection'].apply(lambda x: type(x).__name__)

# Podemos observar que estan anidados como string, en vez de dict, esto dificulta saber si son strings o diccionarios realmente para su transformacion
df[['belongs_to_collection_type','belongs_to_collection']].head(4)

,belongs_to_collection_type,belongs_to_collection
0,str,"{'id': 10194, 'name': 'Toy..."
1,NoneType,None
2,str,"{'id': 119050, 'name': 'Gr..."
3,NoneType,None


In [490]:
import ast

# Función para convertir  str en diccionarios en caso de serlo.
def convert_belongs_dict(val: any) -> dict | str | None:
    """
    Convierte un posible diccionario con (type:str), A su forma verdadera (type:dict).
    Si la fila de entrada es NaN, devuelve None. Si el valor ya es un diccionario, lo devuelve
    sin cambios. Para entradas str, intenta analizarlas como diccionario. Si no es un diccionario ni un nulo, devuelve su forma original de str.

    Parameters:
    --------
    val: Any
        El valor a verificar y convertir, puede ser str, dict o Nan.

    Returns:
    --------
    dict |  str | None:
        dict si la conversión es exitosa, str si no puede convertirse en diccionario, None si ya era nulo.
    """
    
    
    if pd.isna(val):
        return None

    if isinstance(val, str): 
        try:
            # Reemplazar 'Null' por 'None' para que sea un dict válido
            val = val.replace('Null', 'None')

            # Asegurarse de que el valor empiece y termine con { }
            if val.startswith('{') and val.endswith('}'):
                # Evaluar el str como un diccionario usando ast.literal_eval
                return ast.literal_eval(val)
            
            # Si el valor no es un dict válido, devolver la cadena original
            return val
        except (ValueError, SyntaxError) as e:
            print(f"Error convirtiendo el valor: {val}\nError: {e}")
            return val
    
    return val  # Si ya es un diccionario, lo devolvemos sin cambiar


df['belongs_to_collection'] = df['belongs_to_collection'].apply(convert_belongs_dict)

df['belongs_to_collection_type'] = df['belongs_to_collection'].apply(lambda x: type(x).__name__)

# Mostrar el DataFrame con los tipos corregidos
df[[ 'belongs_to_collection', 'belongs_to_collection_type']].head(4)

,belongs_to_collection,belongs_to_collection_type
0,"{'id': 10194, 'name': 'Toy...",dict
1,None,NoneType
2,"{'id': 119050, 'name': 'Gr...",dict
3,None,NoneType


In [491]:
# Ahora si podemos verificar que no hay strings escondidos o erroneos y que todos son diccionarios en realidad.
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40888
<class 'dict'>         4488
Name: count, dtype: int64

In [492]:
# Desanidar los diccionarios en la columna 'belongs_to_collection'
belong_desanidado = pd.json_normalize(df['belongs_to_collection'])

# Renombrar las columnas agregando el prefijo 'btc_'
belong_desanidado.rename(columns={'id': 'btc_id',
                                  'name':'btc_name',
                                  'poster_path':'btc_poster_path',
                                  'backdrop_path': 'btc_backdrop_path'}, inplace=True)
belong_desanidado.head(4)

,btc_id,btc_name,btc_poster_path,btc_backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT...,/9FBwqcd9IRruEDUrTdcaafOMK...
1,NaN,NaN,NaN,NaN
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUc...,/hypTnLot2z8wpFS7qwsQHW1uV...
3,NaN,NaN,NaN,NaN


In [493]:
df = pd.concat([df, belong_desanidado], axis=1)

# Ya podemos borrar la columna auxiliar, belongs_to_collection_type
df = df.drop(columns=['belongs_to_collection_type'])

### Apliquemos el mismo procedimiento para las demas columnas anidadas. `genres`

In [497]:
# La funcion convert_belongs_dict() no funcionaria en esta columna. ya que son listas de diccionarios y no un solo diccionario.


import json
import pandas as pd

# C
def convert_genres(val: any) -> list | str | None:
    """
    Convierte una cadena que representa una lista de diccionarios en una lista de diccionarios.
    Si la entrada es NaN, devuelve None. Si el valor ya es una lista, lo devuelve sin cambios.
    Para entradas str, intenta analizarla como JSON después de reemplazar comillas simples por comillas dobles.
    Si la cadena no es una lista válida, devuelve su forma original de str.

    Parameters:
    --------
    val: Any
        El valor a verificar y convertir, puede ser str, list o Nan.

    Returns:
    --------
    list | str | None:
        list si la conversión es exitosa, str si no puede convertirse en lista, None si ya era nulo.
    """
    
    if pd.isna(val):
        return None

    if isinstance(val, str):
        try:
            # Reemplazar comillas simples por dobles para un JSON válido
            val = val.replace("'", '"')
            # Convertir la cadena a un objeto JSON (lista de diccionarios)
            converted = json.loads(val)
            if isinstance(converted, list) and all(isinstance(item, dict) for item in converted):
                return converted
            else:
                # Si el JSON no es una lista de diccionarios, devolver la cadena original
                return val
        except (ValueError, json.JSONDecodeError) as e:
            print(f"Error convirtiendo el valor: {val}\nError: {e}")
            return val
    
    return val  # Si ya es una lista, lo devolvemos sin cambiar

# Aplicar la función a la columna 'genres'
df['genres'] = df['genres'].apply(convert_genres)

In [498]:
df['genres'].apply(type).value_counts() 

genres
<class 'list'>        45376
<class 'NoneType'>       88
Name: count, dtype: int64

In [495]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45464 entries, 0 to 45338
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  float64       
 2   genres                 45376 non-null  object        
 3   id                     45376 non-null  float64       
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  float64       
 7   production_companies   45376 non-null  object        
 8   production_countries   45376 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status